In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from gprm import ReconstructionModel
import pygmt
import geopandas as gpd
import sys
sys.path.append('/Users/simon/GIT/degenerative_art/')
import map_effects as me
import xarray as xr
from xrspatial import proximity
import gprm.utils.paleogeography as pg
import pygplates
from gprm.utils.create_gpml import gpml2gdf


%load_ext autoreload
%autoreload 2


In [ ]:
Atlantis = ReconstructionModel()

Atlantis.add_continent_polygons('/Users/simon/Documents/2022IMAS-OUC_SOMG/PracFiles/Atlantis2/Atlantis2_continents.gpml')
Atlantis.add_dynamic_polygons('/Users/simon/Documents/2022IMAS-OUC_SOMG/PracFiles/Atlantis2/Atlantis2_topologies.gpml')
Atlantis.add_dynamic_polygons('/Users/simon/Documents/2022IMAS-OUC_SOMG/PracFiles/Atlantis2/Atlantis2_geometries.gpml')
Atlantis.add_rotation_model('/Users/simon/Documents/2022IMAS-OUC_SOMG/PracFiles/Atlantis2/Atlantis2_rotations_rel.rot')

final_grd_sampling = 0.5


In [ ]:
### 2 run make mountain ranges

reconstruction_model = Atlantis
reconstruction_time = 0.
min_distance_to_coastlines=0
max_distance_to_trenches=1200000
sampling = final_grd_sampling


def get_paleogeography_rasters(reconstruction_model, reconstruction_time):
    
    rp = me.reconstruct_and_rasterize_polygons(reconstruction_model.continent_polygons[0], 
                                               reconstruction_model.rotation_model, 
                                               reconstruction_time=reconstruction_time, 
                                               sampling=sampling)

    _, prox_ocean = me.raster_buffer(rp, inside='both')

    # Get subduction zones, and compute a raster of distances to the nearest one
    snapshot = reconstruction_model.plate_snapshot(reconstruction_time)

    szs = snapshot.get_boundary_features(boundary_types=['subduction'])
    all_sz_points = []
    for sz in szs:
        if sz.get_geometry():
            all_sz_points.extend(sz.get_geometry().to_tessellated(np.radians(0.1)).to_lat_lon_list())

    prox_sz = me.points_proximity(x=[lon for lat,lon in all_sz_points],
                                  y=[lat for lat,lon in all_sz_points],
                                  spacing=sampling)

    # combine the rasters to isolate areas not too near to coastlines
    # and not too far from subduction zones
    m1 = prox_ocean.where((prox_ocean>min_distance_to_coastlines))
    m2 = prox_sz.where(prox_sz<max_distance_to_trenches)
    m2.data = m1.data+m2.data
    #mountain_grid = m2

    m2.data[np.isnan(m2.data)] = -999

    mountain_core = proximity(m2, target_values=[-999], distance_metric='GREAT_CIRCLE')


    land = prox_ocean.where(prox_ocean==0, 1)

    #topography = (mountain_core/200)+(land*200)
    #topography = topography.where(topography>0, np.nan)

    return mountain_core, land

mtn, lnd = get_paleogeography_rasters(Atlantis, 150.)

lnd.plot()

In [ ]:
import numpy as np
import xarray as xr
from scipy.interpolate import interpn

def interpolate_values(dataarray, coordinates):
    """
    Interpolate values from a 2D xarray DataArray at arbitrary coordinates using linear interpolation.

    Parameters:
    dataarray (xarray.DataArray): 2D DataArray with coordinates as longitude and latitude.
    points (list of tuples): List of (longitude, latitude) coordinates where interpolation is desired.

    Returns:
    np.ndarray: Interpolated values at the specified points.
    """
    # Extract the coordinates and data values from the DataArray
    lon = dataarray['x'].values
    lat = dataarray['y'].values
    values = dataarray.values

    # Create the grid of points from the coordinates
    grid = (lat, lon)

    points = [(row.geometry.x, row.geometry.y) for i,row in coordinates.iterrows()]
    
    # Perform the interpolation
    interpolated_values = interpn(grid, values, points, method='nearest', bounds_error=False, fill_value=None)

    return interpolated_values
    

import numpy as np
from scipy.stats import truncnorm

def generate_random_coordinates(n_points, central_lat=20, lat_std=5, lon_range=(-200, 200)):
    """
    Generate random coordinates on the surface of a sphere with latitude following a Gaussian distribution.
    
    Parameters:
    n_points (int): Number of random points to generate.
    central_lat (float): Central latitude for Gaussian distribution (in degrees).
    lat_std (float): Standard deviation for the Gaussian distribution (in degrees).
    lon_range (tuple): Range of longitude values (in degrees), typically (0, 360).
    
    Returns:
    list of tuples: List containing the generated (latitude, longitude) pairs.
    """
    # Define the truncated normal distribution for latitude
    lat_min, lat_max = central_lat - (lat_std*3), central_lat + (lat_std*3)  # 15 degrees above and below the central latitude
    a, b = (lat_min - central_lat) / lat_std, (lat_max - central_lat) / lat_std
    lat_distribution = truncnorm(a, b, loc=central_lat, scale=lat_std)
    
    # Generate latitudes
    latitudes = lat_distribution.rvs(n_points)
    
    # Randomly assign half of the points to the southern hemisphere
    southern_hemisphere = np.random.choice([True, False], size=n_points)
    latitudes[southern_hemisphere] *= -1
    
    # Generate longitudes uniformly
    longitudes = np.random.uniform(lon_range[0], lon_range[1], n_points)
    
    # Combine latitudes and longitudes
    #coordinates = list(zip(longitudes, latitudes))
    
    df = gpd.pd.DataFrame(data={'Longitude': longitudes,
                                'Latitude': latitudes})
    return gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude), crs=4326)
    

def generate_climate_sensitive_deposits(land_raster, central_lat, lat_std, n_points=100):

    coordinates = generate_random_coordinates(n_points, central_lat=central_lat, lat_std=lat_std)

    # Interpolate values at the specified points
    coordinates['interpolated_values'] = interpolate_values(land_raster, coordinates)
    #coordinates = coordinates.query('interpolated_values>0')
    
    return coordinates


coordinates = generate_climate_sensitive_deposits(lnd, central_lat=90, lat_std=15, n_points=500)

plt.scatter(coordinates.geometry.x, coordinates.geometry.y, c=coordinates['interpolated_values'], marker='.')
plt.axis([-180,180,-90,90])
plt.colorbar()


In [ ]:
import numpy as np
import xarray as xr

def generate_weighted_random_points(dataarray, n_points):
    """
    Generate random points weighted by the values in a 2D xarray DataArray.

    Parameters:
    dataarray (xarray.DataArray): 2D DataArray with likelihood values at each grid node.
    n_points (int): Number of random points to generate.

    Returns:
    list of tuples: List containing the generated (x, y) coordinates.
    """
    # Normalize the DataArray values to probabilities
    values = dataarray.values
    probabilities = values / values.sum()

    # Flatten the probabilities and generate the CDF
    flattened_probabilities = probabilities.ravel()
    cdf = np.cumsum(flattened_probabilities)

    # Generate uniform random samples
    random_samples = np.random.rand(n_points)

    # Map the uniform random samples to the CDF
    random_indices = np.searchsorted(cdf, random_samples)

    # Convert the flat indices back to 2D indices
    y_indices, x_indices = np.unravel_index(random_indices, dataarray.shape)

    # Generate the corresponding x, y coordinates
    x_coords = dataarray['x'].values[x_indices]
    y_coords = dataarray['y'].values[y_indices]

    # Combine x and y coordinates into tuples
    #coordinates = list(zip(x_coords, y_coords))
    
    df = gpd.pd.DataFrame(data={'Longitude': x_coords,
                                'Latitude': y_coords})
    return gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude), crs=4326)



# Example usage
# Create a sample DataArray
data = np.random.rand(100, 100)  # Example data
#dataarray = xr.DataArray(data, dims=['y', 'x'], coords={'y': np.linspace(-50, 50, 100), 'x': np.linspace(-50, 50, 100)})

n_points = 1000
random_points = generate_weighted_random_points(mtn, n_points)

# Print a few sample coordinates
#for coord in random_points[:10]:
#    print(coord)
plt.plot(random_points.geometry.x, random_points.geometry.y, '.')


In [ ]:
extended_plate_polygons = gpml2gdf(pygplates.FeatureCollection('/Users/simon/Documents/2022IMAS-OUC_SOMG/PracFiles/Atlantis2/Atlantis2_extended_continents.gpml'))

def unreconstruct_geodataframe(gdf, reconstruction_model, reconstructed_polygons, reconstruction_time):

    gdf = gdf.overlay(reconstructed_polygons, how='intersection', keep_geom_type=False)
    return reconstruction_model.reconstruct(gdf, reconstruction_time=reconstruction_time, reverse=True)



In [ ]:
reconstruction_model = Atlantis

fig,ax = plt.subplots()

for reconstruction_time in np.arange(200,-1,-20):
    
    print(reconstruction_time)
    mountain_core, land = get_paleogeography_rasters(reconstruction_model, reconstruction_time)

    reconstructed_polygons = reconstruction_model.reconstruct(extended_plate_polygons, reconstruction_time)

    # Evaporites
    gdf = generate_climate_sensitive_deposits(land, central_lat=20, lat_std=5, n_points=50)
    ur_gdf = unreconstruct_geodataframe(gdf, reconstruction_model, reconstructed_polygons, reconstruction_time)
    ur_gdf.plot(ax=ax, color='orange', markersize=2)
    
    # Glacial Deposits
    gdf = generate_climate_sensitive_deposits(land, central_lat=90, lat_std=15, n_points=100)
    ur_gdf = unreconstruct_geodataframe(gdf, reconstruction_model, reconstructed_polygons, reconstruction_time)
    if ur_gdf is not None:
        ur_gdf.plot(ax=ax, color='blue', markersize=5)
    
    # Volcanos
    volcano_points = generate_weighted_random_points(mountain_core, n_points=20)
    ur_gdf = unreconstruct_geodataframe(volcano_points, reconstruction_model, reconstructed_polygons, reconstruction_time)
    ur_gdf.plot(ax=ax, color='darkred', markersize=2)

    #break

plt.axis([-180,180,-90,90])
plt.show()
    

In [ ]:
land.plot()